In [1]:
# GET PARAMETERS
from examples.benchmark import local_get_args,get_inputs,train_on_ds,keep_track_on_model_metrics,get_trial_id
import sys
import os
import pandas as pd

# Get Parent folder : 
current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)


def get_ds(model_name,dataset_names,dataset_for_coverage,vision_model_name = None):
    save_folder = None
    df_loss,df_results = pd.DataFrame(),pd.DataFrame()
    modification = {'epochs' : 50, #100,
                    }

    # Tricky but here we net to set 'netmob' so that we will use the same period for every combination
    args,folds,hp_tuning_on_first_fold = local_get_args(model_name,
                                                            dataset_names=dataset_names,
                                                            dataset_for_coverage=dataset_for_coverage,
                                                            modification = modification)
    trial_id = get_trial_id(args,vision_model_name=vision_model_name)
    K_fold_splitter,K_subway_ds,dic_class2rpz = get_inputs(args,vision_model_name,folds)
    ds = K_subway_ds[0]
    return(ds,args,trial_id,save_folder,dic_class2rpz,df_loss)

Training and Hyper-parameter tuning with Ray is not possible


In [ ]:
dataset_names = ["subway_in"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'

ds,args,trial_id,save_folder,dic_class2rpz,df_loss = get_ds(model_name,dataset_names,dataset_for_coverage)

### Train :

In [2]:
trainer,df_loss = train_on_ds(model_name,ds,args,trial_id,save_folder,dic_class2rpz,df_loss)

Model size: 0.001GB
number of total parameters: 207233
number of trainable parameters: 207233

start training
epoch: 0 
 min\epoch : 0.10
Estimated time for training: 3.1min 

Training Throughput:498.29 sequences per seconds
>>> Training complete in: 0:02:50.828783
>>> Training performance time: min 0.05649924278259277 avg 0.0617525577545166 seconds (+/- 0.008291510705506254)
>>> Loading performance time: min 0.00014519691467285156 avg 0.03377143307472297 seconds (+/- 0.04339811022797744)
>>> Forward performance time: 0.0445452624313585 seconds (+/- 0.007849904771106716)
>>> Backward performance time: 0.012824211572325439 seconds (+/- 0.0020523130699849797)
>>> Plotting performance time: 8.407904177295918e-06 seconds (+/- 3.453293158395478e-05)
>>> Saving performance time: 0.339694791369968 seconds (+/- 0.08247173046019005)
>>> PI-tracking performance time: 5.473895948760363e-06 seconds (+/- 6.612507479911644e-06)
>>> Scheduler-update performance time: 3.0702474165935904e-06 seconds (+

### Test model :

In [3]:
Preds,Y_true,T_labels = trainer.testing(ds.normalizer, training_mode = 'test')

### Evaluate on non recurrent event: 

#### Specify the spatial unit we want to evaluate 

In [4]:
from build_inputs.preprocess_subway_15 import get_trigram_correspondance

station = 'GER'

df_correspondance = get_trigram_correspondance()
Metro_B_stations = ['Charpennes','Brotteaux','Part-Dieu' ,'Place Guichard', 'Saxe - Gambetta',
       'Jean Macé','Place Jean Jaurès','Debourg','Stade de Gerland',"Gare d'Oullins"]
Metro_B_TRG = list(df_correspondance.set_index('Station').loc[Metro_B_stations].values.reshape(-1))

# Get df_True Volume: 
spatial_units = ds.spatial_unit[ds.spatial_unit.isin(Metro_B_TRG)]
metro_b_indices = list(spatial_units.index)

df_true = pd.DataFrame(Y_true[:,metro_b_indices,0],columns = spatial_units.values,index = ds.tensor_limits_keeper.df_verif_test.iloc[:,-1])

# Get df Predicted Volume : 
df_prediction = pd.DataFrame(Preds[:,metro_b_indices,0],columns = spatial_units.values,index = ds.tensor_limits_keeper.df_verif_test.iloc[:,-1])

### Visualisation: 
#### Cas `subway_in` stade du Lou Gerland avec un match de Rugby : 

In [5]:
from plotting.TS_analysis import drag_selection_box,plot_single_point_prediction,plot_prediction_error

from bokeh.plotting import show,output_notebook
from bokeh.layouts import column
from utils.specific_event import rugby_matches

range = 3*60
kick_off_time,match_times = rugby_matches(df_true.index,range)
width = 1500
height = 400

p1 = plot_single_point_prediction(df_true,df_prediction,station,title= 'Trafic Volume Prediction around at "Stade du Lou Gerland" subway station ',kick_off_time=kick_off_time, range=range,width=width,height = height,show = False)
p2 = plot_prediction_error(df_true,df_prediction,station,metrics =['mae','mse','mape'],title = 'Prediction Error',width=1500,height=400,show=False,min_flow = 20)
select = drag_selection_box(df_true,p1,p2,width=width,height=height//3)
output_notebook()
grid = column(p1,p2,select)
show(grid)

/home/rrochas/prediction-validation/plotting/TS_analysis.py:98: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  real = torch.tensor(real).reshape(-1)
/home/rrochas/prediction-validation/plotting/TS_analysis.py:99: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  predict = torch.tensor(predict).reshape(-1)


Loading BokehJS ...

#### Cas `subway_out` stade du Lou Gerland avec un match de Rugby : 

#### Cas `subway_in` Laurent Bonnevay Astroballe avec un match de Basket : 

#### Cas `subway_out` Laurent Bonnevay Astroballe avec un match de Basket : 

## Maintenant on va intégrer les données NetMob et voir si elles permettent de réduire l'erreur de prédiction :

In [2]:
dataset_names = ["subway_in","netmob_image_per_station"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'
vision_model_name =  'ImageAvgPooling'

ds,args,trial_id,save_folder,dic_class2rpz,df_loss = get_ds(model_name,dataset_names,dataset_for_coverage,vision_model_name=vision_model_name)
trainer,df_loss = train_on_ds(model_name,ds,args,trial_id,save_folder,dic_class2rpz,df_loss)
Preds,Y_true,T_labels = trainer.testing(ds.normalizer, training_mode = 'test')

40
>>>>Model: STGCN; K_fold = 6; Loss function: MSE 

Init Dataset:  torch.Size([7393, 40])
Number of Nan Value:  tensor(0)
Total Number of Elements:  295720 



>>>> Train/Valid/Test split method : similar_length_method

U size:  torch.Size([6239, 40, 7]) Utarget size:  torch.Size([6239, 40, 1])
U_train size:  torch.Size([3743, 40, 7]) Utarget_train size:  torch.Size([3743, 40, 1])
U_valid size:  torch.Size([1248, 40, 7]) Utarget_valid size:  torch.Size([1248, 40, 1])
U_test size:  torch.Size([1247, 40, 7]) Utarget_test size:  torch.Size([1247, 40, 1])
U_train min:  tensor(0.) U_train max:  tensor(10798.)
U_valid min:  tensor(0.) U_valid max:  tensor(1405.)
U_test min:  tensor(0.) U_test max:  tensor(1352.)
Transfer Modes: DL

Init NetMob Dataset:  torch.Size([7392, 40, 1, 6, 6])
Number of Nan Value:  tensor(0)
Total Number of Elements:  10644480 

>>>> Train/Valid/Test split method : similar_length_method

U size:  torch.Size([6238, 40, 1, 6, 6, 7]) Utarget size:  torch.Size([6238, 40, 1, 6, 6, 1])
U_train size:  torch.Size([3742, 40, 1, 6, 6, 7]) Utarget_train size:  torch.Size([3742, 40, 1, 6, 6, 1])
U_valid size:  torch.Size([1248, 40, 1, 

NameError: name 'blabla' is not defined